In [73]:
import pandas as pd
import numpy as np
import csv
import nltk
import xml.etree.ElementTree as ET
from nltk.util import everygrams

import re

In [6]:
# import parser
import spacy 
from spacy.matcher import PhraseMatcher 
nlp = spacy.load('en_core_web_sm')
matcher = PhraseMatcher(nlp.vocab)

# terms would be each list in the dict

In [8]:
import json 

with open ('2019_public_comment.json') as f1:
    dic = json.load(f1)

In [1]:
# dic

In [10]:
## loser case the category
## because annotator use lower case more than upper case
l_topics=['economic', 'environment', 'foreign policy', 
            'governance', 'immigration', 'military', 'safety',
            'social programs', 'social and cultural']

topics=['Economic', 'Environment', 'Foreign policy', 
            'Governance', 'Immigration', 'Military', 'Safety',
            'Social Programs', 'Social and Cultural']
for new,old in zip(l_topics,topics):
    dic[new] = dic[old]
    del dic[old]

In [139]:
df1 = pd.read_excel('500_ads_final_cleaned.xlsx')
# df1 = pd.read_excel('Facebook Ads Analysis.xlsx') # this data is coming from the facebook ad 2016 (the )

In [140]:
df1 = df1[['ad_creative_body','Gold Label']]

In [141]:
df1['Gold Label']=df1['Gold Label'].apply(lambda x:','.join(sorted(x.lower().split(','))))

In [142]:
def find_words(x,lexicon):
    topics=['economic', 'environment', 'foreign policy', 
            'governance', 'immigration', 'military', 'safety',
            'social programs', 'social and cultural']
    
    
    doc = nlp(x)
    words= []
    for t in topics:
        terms= lexicon[t]
        patterns = [nlp.make_doc(text) for text in terms]
        matcher.add("TerminologyList", None, *patterns)
        matches = matcher(doc)
        for match_id, start,end in matches:
            span = doc[start:end]
            words.append(span.text)
    words = list(set(words))
            
    return ','.join(words)
        

In [143]:
# find the words in each message
def find_topic(x,lexicon):
    topics=['economic', 'environment', 'foreign policy', 
            'governance', 'immigration', 'military', 'safety',
            'social programs', 'social and cultural']
    
    if x=='':
        return ''
    
    if x != '': 
        words = x.split(',')
        labels = []
        
        for t in topics:            
            terms = lexicon[t]
            if set(words)&set(terms):
                #print(t)
                #print(set(words)&set(terms))
                labels.append(t)
                
                #l = sorted(labels)
        
        return  ','.join(sorted(labels))
                
        #return ','.join(labels)
            

In [144]:
# get the new labels from classifer2 which depending on the updated lexicon
df1['words'] = df1['ad_creative_body'].apply(lambda x: find_words(x,dic))

#df1.rename(columns={'m_label':'words'},inplace=True)

In [145]:
df1.head(3)

,ad_creative_body,Gold Label,words
0,"Last quarter, Bernie Sanders, Pete Buttigieg a...",none,
1,Every one of the Democratic candidates remaini...,none,"broad coalition,coalition,reading"
2,A young woman came up to me after my speech in...,social programs,"student,college,working people,bank,young woman"


In [146]:
df1['m_label'] = df1['words'].apply(lambda x: find_topic(x,dic))

In [147]:
df1['m_label'] = df1['m_label'].apply(lambda x: 'none' if x=='' else x)

In [148]:
df1.head(10)

,ad_creative_body,Gold Label,words,m_label
0,"Last quarter, Bernie Sanders, Pete Buttigieg a...",none,,none
1,Every one of the Democratic candidates remaini...,none,"broad coalition,coalition,reading","foreign policy,social programs"
2,A young woman came up to me after my speech in...,social programs,"student,college,working people,bank,young woman","economic,social and cultural,social programs"
3,"Como presidenta, Elizabeth proporcionarн_a el ...",social programs,Social Security,social programs
4,Una cosa queda clara: ahorrar para una jubilac...,social programs,"income,Social Security,retirement","economic,social programs"
5,думдуЦI think it is really important that as w...,none,prosecuting,governance
6,As I travel the country talking to workers at ...,economic,"workers,fed",economic
7,думThe senator praised unions as a force for A...,"social programs,economic",healthcare,social programs
8,дум[Her plan] also highlights the need for a n...,"environment, governance, social programs,econ...","water,economic,incomes,tax,health insurance,we...","economic,environment,social programs"
9,думдуЦThe burdens of student debt are not dist...,"social and cultural,social programs","student debt,student",social programs


### Make the matrix and calculate precision, recall F1

In [150]:
df2 = df1[['Gold Label', 'm_label']]

In [151]:
pred_label = ['economic_pred', 'environment_pred', 'foreign policy_pred', 'governance_pred', 
              'immigration_pred', 'military_pred', 'safety_pred','social programs_pred', 
              'social and cultural_pred']

In [152]:
data0 = {x:np.zeros(500,dtype=int) for x in l_topics}
data1 = {x:np.zeros(500,dtype=int) for x in pred_label}

table0= pd.DataFrame(data0)

tabel1 = pd.DataFrame(data1)

c_df = pd.concat([df2,table0,tabel1],axis=1)

c_df.head(10)

,Gold Label,m_label,economic,environment,foreign policy,governance,immigration,military,safety,social programs,social and cultural,economic_pred,environment_pred,foreign policy_pred,governance_pred,immigration_pred,military_pred,safety_pred,social programs_pred,social and cultural_pred
0,none,none,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,none,"foreign policy,social programs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,social programs,"economic,social and cultural,social programs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,social programs,social programs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,social programs,"economic,social programs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,none,governance,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,economic,economic,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,"social programs,economic",social programs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,"environment, governance, social programs,econ...","economic,environment,social programs",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,"social and cultural,social programs",social programs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [174]:
 # in Ben and Juliar's annotation, they made some spelling mistakes of the labels like,
# typing the "social programs" as "social program", type "environment" as "environmental" etc.
# so I made this excel and correct the spelling manually.
# next step is labeling the matrix, so you may got key error because of the spelling mistakes

c_df.to_excel('calculate_f1_500ad.xlsx',index=False)

In [162]:
c_df = pd.read_excel('calculate_f1_500ad.xlsx')

In [163]:
for index, row in c_df.iterrows():
    #print(index)
    #print(row)
    
    if len(row['Gold Label'].split(',')) ==1 and row['Gold Label']!= 'none':
        c_df[row['Gold Label'].strip()][index]=1
        
    if len(row['Gold Label'].split(','))>1:
        for w in row['Gold Label'].split(','):
            #print(index)
            c_df[w.strip()][index]=1
    
    if len(row['m_label'].split(',')) ==1 and row['m_label']!= 'none':
        c_df[row['m_label']+'_pred'][index]=1
        
    if len(row['m_label'].split(','))>1:
        for w in row['m_label'].split(','):
            c_df[w+'_pred'][index]=1
    else: 
        pass

C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarnin

In [164]:
c_df.head()

,Gold Label,m_label,economic,environment,foreign policy,governance,immigration,military,safety,social programs,social and cultural,economic_pred,environment_pred,foreign policy_pred,governance_pred,immigration_pred,military_pred,safety_pred,social programs_pred,social and cultural_pred
0,none,none,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,none,"foreign policy,social programs",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,social programs,"economic,social and cultural,social programs",0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1
3,social programs,social programs,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,social programs,"economic,social programs",0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0


In [169]:
c_df.drop(columns=['Gold Label', 'm_label'],inplace=True)

In [170]:
def confusion_matrix1(vectorg, vectpre):
    tp = 0 
    tn = 0 
    fp = 0
    fn = 0
    for i in range(len(vectorg)):
        if vectorg[i] == 1 and vectpre[i] == 1:
            tp = tp + 1
        elif vectorg[i] == 1 and vectpre[i] == 0:
            fn = fn + 1
        elif vectorg[i] == 0 and vectpre[i] == 0:
            tn = tn + 1
        elif vectorg[i] == 0 and vectpre[i] == 1:
            fp = fp + 1
    print('tp =', tp ,'fp =', fp ,'tn =',tn,'fn =', fn)
    print(tp+fn)
    return (tp, fp, tn, fn)
        
def precision_recall(vectorg, vectpre):
    #print(vectorg[0],vectpre[0])
    tp, fp, tn, fn = confusion_matrix1(vectorg, vectpre)
    precision = tp/(tp + fp)
    print ('precision = ' + str(precision))
    recall = tp/ (tp + fn)
    print ('recall = ' + str(recall))
    f1 = 2/((1/precision)+(1/recall))
    print ('f1 =' + str(f1))
    

In [171]:
categories = list(c_df.columns.values)
categories = categories[0:9]        
print (categories)


['economic', 'environment', 'foreign policy', 'governance', 'immigration', 'military', 'safety', 'social programs', 'social and cultural']


In [173]:
for category in categories:
    x = c_df.columns.get_loc(category)
    print(category.strip())
    precision_recall(c_df.iloc[:,x],c_df.iloc[:,x+9])

economic
tp = 79 fp = 68 tn = 343 fn = 10
89
precision = 0.5374149659863946
recall = 0.8876404494382022
f1 =0.6694915254237288
environment
tp = 29 fp = 15 tn = 452 fn = 4
33
precision = 0.6590909090909091
recall = 0.8787878787878788
f1 =0.7532467532467532
foreign policy
tp = 6 fp = 31 tn = 460 fn = 3
9
precision = 0.16216216216216217
recall = 0.6666666666666666
f1 =0.2608695652173913
governance
tp = 17 fp = 13 tn = 442 fn = 28
45
precision = 0.5666666666666667
recall = 0.37777777777777777
f1 =0.4533333333333333
immigration
tp = 8 fp = 5 tn = 486 fn = 1
9
precision = 0.6153846153846154
recall = 0.8888888888888888
f1 =0.7272727272727273
military
tp = 7 fp = 5 tn = 487 fn = 1
8
precision = 0.5833333333333334
recall = 0.875
f1 =0.7000000000000001
safety
tp = 24 fp = 22 tn = 448 fn = 6
30
precision = 0.5217391304347826
recall = 0.8
f1 =0.631578947368421
social programs
tp = 64 fp = 19 tn = 409 fn = 8
72
precision = 0.7710843373493976
recall = 0.8888888888888888
f1 =0.8258064516129032
social

In [49]:
# from sklearn.metrics import precision_recall_fscore_support
# # from sklearn.metrics import  f1_score
# y_true  = np.array(c_df['y_test'].tolist())
# y_pred = np.array(c_df['y_predict'].tolist())
# precision_recall_fscore_support(y_true, y_pred, average=None ,  labels = [1,0])

(array([1., 0.]),
 array([0.536, 0.   ]),
 array([0.69791667, 0.        ]),
 array([500,   0], dtype=int64))

In [85]:
## calculate how many contains the right label, include none
right_index = []
for index, row in c_df.iterrows():
    gl = row['Gold Label'].split(',')
    ml = row['m_label'].split(',')
    
    if set(gl)&set(ml):
        right_index.append(index)

In [86]:
df1.loc[right_index].shape  
## 422 messages can be detected the right label 

(422, 4)

In [87]:
## calculate the kappa score
from sklearn.metrics import cohen_kappa_score

cohen_score1 = cohen_kappa_score(c_df['Gold Label'].astype(str), c_df['m_label'].astype(str))
print("aggrement of Gold Label and Computational Topic Identification" + str(cohen_score1))

aggrement of Gold Label and Computational Topic Identification0.5040385432903499


In [5]:
# doc = nlp('the TAX PLAN and the tax plan and the Tax Plan is tax planed')
# terms = ['tax plan']
# patterns = [nlp.make_doc(text) for text in terms]
# matcher.add("TerminologyList", None, *patterns)
# matches = matcher(doc)
# for match_id, start,end in matches:
#     span = doc[start:end]
#     print(span.text)  # not work for the lemmatizer 

tax plan
